In [ ]:
import os
import random
import numpy as np
from imutils import paths
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
SOURCE_DIR = os.path.abspath('/data/aumkar/train')
TARGET_DIR = os.path.abspath('/data/aumkar/validation')
DATA_LOAD_DIR = os.path.abspath('/data/aumkar/data_load')

In [ ]:
source_path = list(paths.list_images(SOURCE_DIR))
target_path = list(paths.list_images(TARGET_DIR))

In [ ]:
labels_unique = np.load(os.path.join(DATA_LOAD_DIR, 'labels.npy'))

In [ ]:
clf = linear_model.SGDClassifier(max_iter = 100, tol = 1e-3, n_jobs = -1)

In [ ]:
def lst2arr_reshape(lst, data):
    
    arr = np.asarray(lst)
    
    if data == 'data':
        return arr.reshape((arr.shape[0] * arr.shape[1], arr.shape[2]))
    elif data == 'label':
        return arr.reshape((arr.shape[0] * arr.shape[1]))

In [ ]:
def num_gen(start, end, size): 
    num = [] 
  
    for i in range(size): 
        num.append(random.randint(start, end)) 
  
    return num

In [ ]:
def target_data_batches(size, pred, lab):
    
    '''sampid = np.arange(len(target_path))
    np.random.shuffle(sampid)'''
    
    ids = num_gen(0, 865, size)
    
    x_t = []
    y_t = []
    
    for i in ids:
        
        data_target = np.load(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_target_%s.npz' % i))
        x_t.append(data_target['a'])
        
        if pred == 'first':
                       
            pseudo_label = np.load(os.path.join(DATA_LOAD_DIR, 'inception_predictions', 'incept_pred_%s.npz' % i))
            y_t.append(pseudo_label['a'])
            
        elif pred == 'later':
            
            if i != 865 and i != 0:
                y_t.append(lab[(i * 64) : (i * 64) + 64]) #debug (i or (((i - 1) * 64) + 1))
            elif i == 0:
                y_t.append(lab[:64])
            elif i == 865:
                y_t.append(lab[-64:])
        
    if pred == 'first':
        return lst2arr_reshape(x_t, 'data'), lst2arr_reshape(y_t, 'label')
    elif pred == 'later':
        return lst2arr_reshape(x_t, 'data'), np.asarray(y_t)

In [ ]:
def predictions():
    
    pred = []
    
    for i in range(866):
        
        target_data = np.load(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_target_%s.npz' % i))
        target_data_ = target_data['a'].reshape((target_data['a'].shape[0] * target_data['a'].shape[1], target_data['a'].shape[2]))
        pred.append(clf.predict(target_data_))
        
    return np.asarray(pred)

In [ ]:
def single_sslda(sourcex, sourcey, targetx, targety, M):

    for i in range(1, M):
        
        print('Iteration: ', i)
         
        n = (i / float(M)) * len(target_path)
        
        if i == 1:
            target_sample, target_label = target_data_batches(int(n), 'first', None)
        else:
            target_sample, target_label = target_data_batches(int(n), 'later', fin_pred)
        
        for i in range(2382):
            
            data = np.load(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_%s.npz' % i))
            label = np.load(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_label_%s.npz' % i))
            
            sourcex = data['a'].reshape((data['a'].shape[0] * data['a'].shape[1], data['a'].shape[2]))
            sourcey = label['a'].reshape((label['a'].shape[0] * label['a'].shape[1]))
                  
            concat_sample = np.append(sourcex, target_sample, axis = 0)
            concat_label = np.append(sourcey, target_label, axis = 0)
            
            if i == 0:
                clf.partial_fit(concat_sample, concat_label, classes = labels_unique)
            else:
                clf.partial_fit(concat_sample, concat_label)

        print('Predicting for iteration: ', i)
            
        fin_pred = predictions()
        
    return fin_pred

In [ ]:
sslda_pred = single_sslda(xsource, ysource, xtarget, ytarget, 20)

In [ ]:
classes = ['aeroplane', 'bicycle', 'bus', 'car', 'horse', 'knife', 'motorcycle', 'person', 'plant', 'skateboard', 
          'train', 'truck']

In [ ]:
print(classification_report(ytarget, sslda_pred, target_names = classes))